In [5]:
# train model
from main_segmentation import trainModel
trainModel()

Epoch 1/20
144/144 [==============================] - ETA: 0s - loss: 0.0626 - output1_loss: 0.0195 - output2_loss: 0.0157 - output3_loss: 0.0089 - output_fuse_loss: 0.0185 - output_fuse_ofuse_pixel_error: 0.5474
Epoch 1: val_loss improved from inf to 0.02431, saving model to checkpoints\IHED\checkpoint.01-0.02.hdf5
144/144 [==============================] - 58s 343ms/step - loss: 0.0626 - output1_loss: 0.0195 - output2_loss: 0.0157 - output3_loss: 0.0089 - output_fuse_loss: 0.0185 - output_fuse_ofuse_pixel_error: 0.5474 - val_loss: 0.0243 - val_output1_loss: 0.0063 - val_output2_loss: 0.0069 - val_output3_loss: 0.0058 - val_output_fuse_loss: 0.0053 - val_output_fuse_ofuse_pixel_error: 0.0883
Epoch 2/20
144/144 [==============================] - ETA: 0s - loss: 0.0303 - output1_loss: 0.0076 - output2_loss: 0.0092 - output3_loss: 0.0069 - output_fuse_loss: 0.0066 - output_fuse_ofuse_pixel_error: 0.3174
Epoch 2: val_loss did not improve from 0.02431
144/144 [=============================

In [2]:
# predict
from test import predictResults
%load_ext memory_profiler
%memit predictResults()

--- select weight data ---
0 13-06-2022_400-400_250-20-5.hdf5
1 15-06-2022_400-400_250-20-3.hdf5
2 19-06-2022_800-800_320-20-2.hdf5
in_train_data/image151-1.jpg 800 800
1/1 [==============================] - 0s 207ms/step
in_train_data/image234-1.jpg 800 800
1/1 [==============================] - 0s 18ms/step
in_train_data/image234-2.jpg 800 800
1/1 [==============================] - 0s 20ms/step
in_train_data/image235-1.jpg 800 800
1/1 [==============================] - 0s 19ms/step
in_train_data/image235-2.jpg 800 800
1/1 [==============================] - 0s 20ms/step
in_train_data/image236-0.jpg 800 800
1/1 [==============================] - 0s 17ms/step
not_train_data/image139-1.jpg 800 800
1/1 [==============================] - 0s 17ms/step
not_train_data/image145-1.jpg 800 800
1/1 [==============================] - 0s 19ms/step
not_train_data/image245-2.jpg 800 800
1/1 [==============================] - 0s 18ms/step
not_train_data/image247-1.jpg 800 800
1/1 [====================

In [4]:
# train pair generator
import os
from constant import DATA_FOLDER, BATCH_SIZE
from constant import TRAIN_VALIDATOR_RATIO

# check train/test equality
images = os.listdir(DATA_FOLDER + "/train/images/")
labels = os.listdir(DATA_FOLDER + "/train/labels/")

isError = False
errorList = ["--- image that isn't in labels folder ---"]
for image in images:
    if image not in labels:
        errorList.append(image)
        isError = True
if len(errorList) > 1:
    for error in errorList:
        print(error)

errorList = ["--- label that isn't in images folder ---"]
for label in labels:
    if label not in images:
        errorList.append(label)
        isError = True
if len(errorList) > 1:
    for error in errorList:
        print(error)

if isError:
    raise Exception("images/labels folder didn't match")

# create train_pair
output = ""
toTrainSize = len(images)
while (
    round(toTrainSize * TRAIN_VALIDATOR_RATIO, 5) % (BATCH_SIZE * 2) != 0
    or round(toTrainSize * (1 - TRAIN_VALIDATOR_RATIO), 5) % (BATCH_SIZE * 2) != 0
):
    toTrainSize -= 1

for index, path in enumerate(images):
    if index >= toTrainSize:
        break
    output += "train/images/" + path
    output += " "
    output += "train/labels/" + path
    output += "\n"
print('batch size:',BATCH_SIZE)
print("created", toTrainSize, "pairs out of", len(images), "pairs")

with open(DATA_FOLDER + "/train_pair.lst", "w") as f:
    f.write(output)


batch size: 2
created 320 pairs out of 349 pairs


In [ ]:
# resize image
import os
import glob
from constant import DATA_FOLDER
from PIL import Image

imagesPath = glob.glob("images/*")
NEW_SIZE = (800, 800)

def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
        if size < 1024.0 or unit == 'PB':
            break
        size /= 1024.0
    return f"{size:.{decimal_places}f} {unit}"

for path in imagesPath:
    img = Image.open(path)
    if img.size != NEW_SIZE:
        print(path, ":", img.size, "->", NEW_SIZE, ":", human_readable_size(os.path.getsize(path)), end=" -> ")
        img = img.resize(NEW_SIZE)
        img.save(path, quality=100)
        print(human_readable_size(os.path.getsize(path)))
